# Download Dataset

In [1]:
using NPZ
using Flux
train_ds_x = permutedims(npzread("../dataset/TF/train_x.npy"),[2,3,4,1])
# train_ds_orig_x = permutedims(npzread("../dataset/TF/train_x.npy"),[2,3,4,1])
train_ds_y = hcat([Flux.onehot(idx,0:7) for idx=npzread("../dataset/TF/train_y.npy")]...)
val_ds_x = permutedims(npzread("../dataset/TF/val_x.npy"),[2,3,4,1])
val_ds_y = hcat([Flux.onehot(idx,0:7) for idx=npzread("../dataset/TF/val_y.npy")]...)


8×800 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1  ⋅  1  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  ⋅
 ⋅  1  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅     1  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅

In [2]:
using Statistics
# Compute mean 
ds_mean =  mean(train_ds_x)
ds_sqrt_var = sqrt(var(train_ds_x))

0.76747113f0

In [3]:
# Use the DataLoader
trainingData = Flux.DataLoader((train_ds_x, train_ds_y), batchsize=64,shuffle=true)
valData = Flux.DataLoader((val_ds_x, val_ds_y), batchsize=64)

MLUtils.DataLoader{Tuple{Array{Float32, 4}, Flux.OneHotArray{UInt32, 8, 1, 2, Vector{UInt32}}}, Random._GLOBAL_RNG, Val{nothing}}(([0.005956173 0.012404119 … 6.2214305f-5 0.00013177958; 0.0021701786 0.015968 … 3.723193f-5 7.755589f-6; … ; 0.006898921 0.035660375 … 6.542493f-5 7.087481f-5; 0.012738511 0.030686239 … 5.143952f-5 7.6339114f-5;;;; 0.21398869 0.22579263 … 4.8694346f-5 7.811189f-5; 0.30852592 0.18149962 … 5.682248f-5 4.3034554f-5; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; 0.0029731644 0.081075326 … 6.289727f-5 7.9883495f-5; 0.015109962 0.048517507 … 9.040684f-5 1.857942f-5; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;;; … ;;;; 0.0023676283 0.0038272867 … 5.4099914f-5 4.6927482f-5; 0.003977077 0.0033666817 … 5.9143527f-5 4.938268f-5; … ; 0.0009947252 0.008146661 … 0.00013018295 7.9826044f-5; 0.00020582468 0.002836615 … 0.00014470241 0.00017577963;;;; 0.0010776172 0.00031317244 … 0.00017337126 0.00018267008; 0.0011409046 0.00097584416 … 9.5102405f-5 0.00012270035; … ; 0.000920

In [5]:
# Model Definition

model = Chain(
    Upsample(:bilinear,size=(32,32)),
    x->(x .- ds_mean) ./ ds_sqrt_var,
    Conv((3,3),1=>32, Flux.relu),
    Conv((3,3),32=>64, Flux.relu),
    MaxPool((2,2)),
    Dropout(0.25),
    Flux.flatten,
    Dense(12544=>128,relu),
    Dropout(0.5),
    Dense(128=>8),
)
# Checking 
model(train_ds_x[:,:,:,1:1])

8×1 Matrix{Float32}:
  0.02195992
  0.035274234
 -0.012831309
 -0.016803723
 -0.043840878
 -0.030042108
  0.03075295
 -0.07204841

In [6]:
# Loss Function
function loss(x,y)
    ŷ = model(x)
    Flux.Losses.logitcrossentropy(ŷ,y)
end

# Check Accuracy
function getAccuracy(m,d)
    acc = 0
    for (x,y)=d
        ŷ = model(x)
        acc += sum(Flux.onecold(ŷ) .== Flux.onecold(y)) / size(x)[end]
    end
    acc/length(d)
end

# Total Loss
function loss_tot(d)
    l = 0
    for (x,y) = d
        l+= loss(x,y)
    end
    return l/length(d)
end

loss_tot (generic function with 1 method)

In [7]:
# Goes very slow :''(
#@show loss_tot(valData) 
testmode!(model,true)
@show getAccuracy(model,valData)

getAccuracy(model, valData) = 0.15264423076923078

0.15264423076923078

In [8]:
# Does not Learn
opt = Flux.Optimise.ADAM()
ps = Flux.params(model)
testmode!(model,false)

evalcb() = println("Loss: $(loss_tot(valData))")

Flux.@epochs 10 Flux.train!(
    loss,
    ps,
    trainingData,
    opt,
    cb=Flux.throttle(evalcb,10),
)

testmode!(model,true)
@show getAccuracy(model,valData)


┌ Info: Epoch 1
└ @ Main C:\Users\detter55\.julia\packages\Flux\js6mP\src\optimise\train.jl:154


Loss: 2.1690462


Loss: 1.6973624


Loss: 1.3927549


┌ Info: Epoch 2
└ @ Main C:\Users\detter55\.julia\packages\Flux\js6mP\src\optimise\train.jl:154


...

Loss: 0.69252163


Loss: 0.6282399


Loss: 0.6058708


getAccuracy(model, valData) = 0.8569711538461539


0.8569711538461539

In [10]:
compute_source_index_and_lambda

UndefVarError: UndefVarError: compute_source_index_and_lambda not defined